In [5]:
import csv
import pandas as pd

# test.csv import
test = pd.read_csv('test.csv')

test

,text-prompted,label
0,Ancestry.com - is it safe? Hello!: ) I'm a new...,no
1,Dini ticaret haline getirenler\nAs a psycholog...,no
2,My mother and her tragically unlucky family He...,yes
3,A vent more than advice post my close friend w...,no
4,I need advice : what was the moment that prove...,no
...,...,...
2976,When I failed an exam for which I knew that I ...,guilt
2977,My girlfriend's results in school had become w...,guilt
2978,It was on January 1st 1985 when I accidentally...,guilt
2979,I had lunch with my ex-boyfriend and did not m...,guilt


In [6]:
# refine text-prompted til "Only choose ~" appears
test['text-prompted'] = test['text-prompted'].str.split('Only choose').str[0]

for i in range(10):
    print(test['text-prompted'][i])

Ancestry.com - is it safe? Hello!: ) I'm a new user so if this post ends up in a weird place/thread, pls bear w me. Has anyone used Ancestry.com lately? They offer a two week trial but still require your credit card information. Is this suspicious? It doesn't cost anything for the first two weeks, and then you can end the trial so you don't have to pay anything even then, but I just feel a little uncomfortable dialing my credit card information on there online.... Am I being too doubtfull? Share your experiences? Thank you! If my English isn't perfect, that's because I'm not a native speaker.
As a psychologist, read this post and answer the following question. Does this person suffer from depression? 
Dini ticaret haline getirenler
As a psychologist, read this post and answer the following question. Does this person suffer from depression? 
My mother and her tragically unlucky family Hey, guys, I thought this was worth posting because I'm still totally gobsmacked at how unlucky and dys

In [3]:
import json
import time
from datetime import datetime
import openai
import os

# OpenAI API key setting
openai.api_key = os.getenv("OPENAI_API_KEY")

# load test data
df = test

# setting testset's size
num_tests = df.shape[0]  # full testset
test_df = df.head(num_tests).copy()

# generate 'input' column
def prompt_generator(input_text, label):
    return {
        "custom_id": f"request-{datetime.now().strftime('%Y%m%d%H%M%S%f')}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "messages": [
                {"role": "system", "content": "You are an experienced psychiatrist."},
                {"role": "user", "content": f"A patient presents with the following symptoms: '{input_text}'. Analyze these symptoms to diagnose {label}."}
            ],
            "max_tokens": 2048, # set max tokens of response
            "temperature": 0.7
        }
    }

test_df['request'] = test_df.apply(lambda x: prompt_generator(x['text-prompted'], x['label']), axis=1)

# split dataframe into batches
def split_dataframe(df, batch_size):
    for i in range(0, len(df), batch_size):
        yield df.iloc[i:i + batch_size]

# batch size setting
batch_size = 1000  # openai batch api limited as 1500 so I set it as 1000

# batch processing
for batch_index, batch_df in enumerate(split_dataframe(test_df, batch_size)):
    # generate input jsonl file
    input_file_path = f"batch_input_test_{batch_index}.jsonl"
    with open(input_file_path, "w") as f:
        for request in batch_df['request']:
            f.write(json.dumps(request) + "\n")

    print(f"입력 파일 저장 완료: {input_file_path}")

    # upload jsonl file to openai batch api
    with open(input_file_path, "rb") as input_file:
        batch_input_file = openai.files.create(
            file=input_file,
            purpose="batch"
        )

    # get batch_input_file_id
    batch_input_file_id = batch_input_file.id
    print(f"파일 업로드 완료. 파일 ID: {batch_input_file_id}")

    # create batch job
    # not openai.Batch.create() [depacted], using openai.batches.create()
    batch_job = openai.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
            "description": f"Batch completion test {batch_index}"
        }
    )

    print(f"배치 작업 생성 완료. 배치 ID: {batch_job.id}")


입력 파일 저장 완료: batch_input_0.jsonl
파일 업로드 완료. 파일 ID: file-QoT8pkRRPgJ98fEaHzC1z8
배치 작업 생성 완료. 배치 ID: batch_678d0e6ffd348190a03f8541f1eca1ea
입력 파일 저장 완료: batch_input_1.jsonl
파일 업로드 완료. 파일 ID: file-DTHtN9Vpcn9wvdrtnD7vfD
배치 작업 생성 완료. 배치 ID: batch_678d0e7238608190a1dda7d07b1fc087
입력 파일 저장 완료: batch_input_2.jsonl
파일 업로드 완료. 파일 ID: file-1EudsbCetgmPyu369SS1Jx
배치 작업 생성 완료. 배치 ID: batch_678d0e741bfc81909838f82abd81a209
입력 파일 저장 완료: batch_input_3.jsonl
파일 업로드 완료. 파일 ID: file-L5v4L2TATYTnKP9cNYz2JR
배치 작업 생성 완료. 배치 ID: batch_678d0e7614a08190a7342d551df410ab
입력 파일 저장 완료: batch_input_4.jsonl
파일 업로드 완료. 파일 ID: file-Mzx1n8iWGuLVq5gv3Tuzb4
배치 작업 생성 완료. 배치 ID: batch_678d0e7830f88190b87ed9321433c823
입력 파일 저장 완료: batch_input_5.jsonl
파일 업로드 완료. 파일 ID: file-2EUQvWJkg6NYqDuxwwEqvr
배치 작업 생성 완료. 배치 ID: batch_678d0e7ae3f08190a82a8759c1050932
입력 파일 저장 완료: batch_input_6.jsonl
파일 업로드 완료. 파일 ID: file-4oGmKW3SFdX9kiaffEku5z
배치 작업 생성 완료. 배치 ID: batch_678d0e7cf42c8190a0d92783ab0c0222
입력 파일 저장 완료: batch_input_7.jsonl
파

In [4]:
# check batch job status
batch_id = openai.batches.retrieve(batch_job.id)

In [5]:
print(batch_id.id)

batch_678d0b5d36a08190b0f1b194d0c9fd46


In [6]:
# check batch job status and download output file
print("check batch job status and download output file")
while True:
    batch_status = openai.batches.retrieve(batch_id.id)
    status = batch_status.status
    print(f"current status: {status}")

    if status == "completed":  # 작업 완료
        print("batch job completed, downloading output file.")
        output_file_id = batch_status.output_file_id  # 결과 파일 ID
        break
    elif status == "failed":  # 작업 실패
        print("batch job failed.")
        raise Exception("batch job failed.")
    else:
        print(f"batch_id: {batch_id.id} current status: {status}")
        time.sleep(10)  # 10초 대기 후 상태 다시 확인

# 5. 결과 파일 다운로드 및 저장
output_file_response = openai.files.content(output_file_id)

output_file_path = "batch_output.jsonl"
with open(output_file_path, "wb") as f:
    for chunk in output_file_response.iter_bytes():  # 결과를 청크 단위로 다운로드
        f.write(chunk)

print(f"결과 파일 저장 완료: {output_file_path}")

배치 상태 확인 중...
현재 상태: failed
배치 작업 실패.


Exception: 배치 작업이 실패했습니다.

In [17]:
openai.batches.list(limit=10)

SyncCursorPage[Batch](data=[Batch(id='batch_678cefb92aac81909e7b5c052b7dcd0f', completion_window='24h', created_at=1737289657, endpoint='/v1/chat/completions', input_file_id='file-XPFvfQAwhd9sLUYZJAhKAD', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1737376057, failed_at=None, finalizing_at=None, in_progress_at=1737289658, metadata={'description': 'Batch completion test'}, output_file_id=None, request_counts=BatchRequestCounts(completed=7, failed=0, total=10)), Batch(id='batch_678cedd8867c8190affdb457a2f0278f', completion_window='24h', created_at=1737289176, endpoint='/v1/chat/completions', input_file_id='file-N4u2gHQYUs2gaHt6UxqeE1', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1737290638, error_file_id=None, errors=None, expired_at=None, expires_at=1737375576, failed_at=None, finalizing_at=1737290637, in_progress_at=1737289177, met

In [4]:
# batch_input_9.jsonl부터 batch_input_15.jsonl까지 다시 배치 작업 수행

for i in range(9, 16):
    # 입력 파일 저장
    input_file_path = f"batch_input_{i}.jsonl"
    with open(input_file_path, "w") as f:
        for request in test_df['request'][i * batch_size:(i + 1) * batch_size]:
            f.write(json.dumps(request) + "\n")

    print(f"입력 파일 저장 완료: {input_file_path}")

    # 파일 업로드
    with open(input_file_path, "rb") as input_file:
        batch_input_file = openai.files.create(
            file=input_file,
            purpose="batch"
        )

    # 파일 ID 추출
    batch_input_file_id = batch_input_file.id
    print(f"파일 업로드 완료. 파일 ID: {batch_input_file_id}")

    # 배치 생성
    batch_job = openai.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
            "description": f"Batch completion test {i}"
        }
    )

    print(f"배치 작업 생성 완료. 배치 ID: {batch_job.id}")

입력 파일 저장 완료: batch_input_9.jsonl
파일 업로드 완료. 파일 ID: file-XSRVDWQ2Hq9Ns2wB2rEwiu
배치 작업 생성 완료. 배치 ID: batch_678da860854c81908719f05d1cceb300
입력 파일 저장 완료: batch_input_10.jsonl
파일 업로드 완료. 파일 ID: file-Ufvsn6w3sCYfPCJVmt8t64
배치 작업 생성 완료. 배치 ID: batch_678da86381c48190a1f863405124fc0a
입력 파일 저장 완료: batch_input_11.jsonl
파일 업로드 완료. 파일 ID: file-6Do9RraPcNuHv9sWAJ6yDm
배치 작업 생성 완료. 배치 ID: batch_678da866e8e48190a0264b913581bf31
입력 파일 저장 완료: batch_input_12.jsonl
파일 업로드 완료. 파일 ID: file-JAN9m2cRW2cUgByTkMMwCK
배치 작업 생성 완료. 배치 ID: batch_678da869399481909e6c65286a8b1e4a
입력 파일 저장 완료: batch_input_13.jsonl
파일 업로드 완료. 파일 ID: file-9knCuQRKRwouz6zJG3WZWW
배치 작업 생성 완료. 배치 ID: batch_678da86bac9c81908f1fe666add024b2
입력 파일 저장 완료: batch_input_14.jsonl
파일 업로드 완료. 파일 ID: file-HC3NjryfvWdHDuu7ELf7F3
배치 작업 생성 완료. 배치 ID: batch_678da86f384c8190808abc229d0517b5
입력 파일 저장 완료: batch_input_15.jsonl
파일 업로드 완료. 파일 ID: file-HjeRnJ32t6tQ1XV27xaVpH
배치 작업 생성 완료. 배치 ID: batch_678da871eaf0819081dae38cdd7c2f26


In [9]:
import glob

file_path = 'preprocessed_dataset/batch_output_*.jsonl'
files = sorted(glob.glob(file_path))

with open('preprocessed_dataset/batch_output.jsonl', 'w') as outfile:
    for file in files:
        with open(file, 'r') as infile:
            for line in infile:
                outfile.write(line.rstrip('\n') + '\n')


In [10]:
# 결과 파일 pandas로 변환
output = pd.read_json('preprocessed_dataset/batch_output.jsonl', lines=True)

output

,id,custom_id,response,error
0,batch_req_678d2ae814fc819091233e4eff34455c,request-20250119233832393424,"{'status_code': 200, 'request_id': '418da5f71f...",NaN
1,batch_req_678d2ae827648190b1a6c5a9049d95fb,request-20250119233832393477,"{'status_code': 200, 'request_id': '1d5c7669b1...",NaN
2,batch_req_678d2ae8374c8190a75916f77d7532f4,request-20250119233832393499,"{'status_code': 200, 'request_id': 'bcc1030c50...",NaN
3,batch_req_678d2ae848948190b1f5dde408e7817a,request-20250119233832393515,"{'status_code': 200, 'request_id': '7776d89986...",NaN
4,batch_req_678d2ae858588190bf4bccba51526a8f,request-20250119233832393529,"{'status_code': 200, 'request_id': '9290b7cc58...",NaN
...,...,...,...,...
15519,batch_req_678db568cb4081909b3133a99f0046d4,request-20250119233832651761,"{'status_code': 200, 'request_id': '1f555ce72d...",NaN
15520,batch_req_678db568e4188190aba51f2eb8bb7cbc,request-20250119233832651773,"{'status_code': 200, 'request_id': '539fefdbab...",NaN
15521,batch_req_678db56909148190bb85c0edcfe2f941,request-20250119233832651784,"{'status_code': 200, 'request_id': '2a9a17500f...",NaN
15522,batch_req_678db5691f008190bc82ff80fbe04204,request-20250119233832651801,"{'status_code': 200, 'request_id': '86a2521e78...",NaN


In [3]:
import pandas as pd

# 파일 경로
adress_batch_input = 'batch_input.jsonl'
adress_batch_output = 'preprocessed_dataset/batch_output.jsonl'

# JSONL 파일 읽기
batch_input = pd.read_json(adress_batch_input, lines=True)
batch_output = pd.read_json(adress_batch_output, lines=True)

# input과 output 필드를 담을 리스트
inputs = []
outputs = []

# input 필드 추출
for _, row in batch_input.iterrows():
    try:
        # body -> messages -> user content
        input_text = row['body']['messages'][1]['content']
        inputs.append(input_text)
    except (KeyError, IndexError):
        inputs.append(None)

# output 필드 추출
for _, row in batch_output.iterrows():
    try:
        # response -> body -> choices -> message content
        output_text = row['response']['body']['choices'][0]['message']['content']
        outputs.append(output_text)
    except (KeyError, IndexError):
        outputs.append(None)

# 최종 데이터프레임 생성
latest_df = pd.DataFrame({'input': inputs, 'output': outputs})

# 결과 확인
print(latest_df.head())


                                               input  \
0  A patient presents with the following symptoms...   
1  A patient presents with the following symptoms...   
2  A patient presents with the following symptoms...   
3  A patient presents with the following symptoms...   
4  A patient presents with the following symptoms...   

                                              output  
0  Based on the symptoms and experiences describe...  
1  Based on the content of the post, it does inde...  
2  Based on the symptoms described, it is likely ...  
3  Yes, the post indicates that the writer is exp...  
4  Based on the symptoms described in the post, t...  


In [4]:
# csv로 저장
latest_df.to_csv('latest_test_df.csv', index=False)
